In [108]:
import re
import pandas as pd
import numpy as np
#from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [109]:
train_sentences = pd.read_csv("../data/dataset_mr/Trainset_complete.csv", sep=';') 
test_sentences = pd.read_csv("../data/dataset_mr/Validationset.csv", sep=';') 

In [110]:
test_sentences.head(10)


,Unnamed: 0,SentenceId,PhraseId,Phrase,Sentiment
0,7096,7108,131841,Michele is a such a brainless flibbertigibbet ...,-1
1,6876,6887,128029,The question hanging over The Time Machine is ...,0
2,7310,7322,135561,Worse than ` Silence of the Lambs ' better tha...,0
3,7002,7014,130241,Ninety minutes of Viva Castro !,0
4,6629,6639,123623,Dong shows how intolerance has the power to de...,0
5,6982,6994,129911,Graham Greene 's novel of colonialism and empi...,1
6,6441,6451,120640,It grabs you in the dark and shakes you vigoro...,1
7,7044,7056,130906,Mushes the college-friends genre -LRB- The Big...,0
8,6544,6554,122262,-LRB- H -RRB- ad I suffered and bled on the ha...,-1
9,6489,6499,121347,Nettelbeck ... has a pleasing way with a metap...,1


In [111]:
s = 0.0
for i in train_sentences['Phrase']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each review : ",s/train_sentences.shape[0])
pos = 0
neg = 0
for i in range(train_sentences.shape[0]):
    if train_sentences.iloc[i]['Sentiment'] == 1:
        pos = pos + 1
    elif train_sentences.iloc[i]['Sentiment'] == -1:
        neg = neg + 1
    
neu = train_sentences.shape[0]-pos-neg
print("Percentage of reviews with positive sentiment is "+str(pos/train_sentences.shape[0]*100)+"%")
print("Percentage of reviews with negative sentiment is "+str(neg/train_sentences.shape[0]*100)+"%")
print("Percentage of reviews with neutral sentiment is "+str(neu/train_sentences.shape[0]*100)+"%")

Average length of each review :  18.783382789317507
Percentage of reviews with positive sentiment is 33.33333333333333%
Percentage of reviews with negative sentiment is 33.33333333333333%
Percentage of reviews with neutral sentiment is 33.33333333333333%


#### Before being fed into the LSTM model, the data needs to be padded and tokenized:
#### Tokenizing: Keras’ inbuilt tokenizer API has fit the dataset, which splits the sentences into words and creates a dictionary of all unique words found and their uniquely assigned integers. Each sentence is converted into an array of integers representing all the individual words present in it.
#### Sequence Padding: The array representing each sentence in the dataset is filled with zeroes to the left to make the size of the array ten and bring all collections to the same length.

In [129]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

An embedding layer of dimension 100 converts each word in the sentence into a fixed-length dense vector of size 100. The input dimension is set as the vocabulary size, and the output dimension is 100. Each word in the input will hence get represented by a vector of size 100.
A bidirectional LSTM layer of 64 units.
A dense (fully connected) layer of 24 units with relu activation.
A dense layer of 1 unit and sigmoid activation outputs the probability of the review is positive, i.e. if the label is 1

In [125]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 200, 100)          300000    
                                                                 
 bidirectional_8 (Bidirecti  (None, 128)               84480     
 onal)                                                           
                                                                 
 dense_16 (Dense)            (None, 24)                3096      
                                                                 
 dense_17 (Dense)            (None, 1)                 25        
                                                                 
Total params: 387601 (1.48 MB)
Trainable params: 387601 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [126]:
train_labels = train_sentences['Sentiment'].values
test_labels = test_sentences['Sentiment'].values
train_labels

array([-1, -1, -1, ..., -1, -1,  0], dtype=int64)

In [131]:
num_epochs = 100
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: -0.3725 - accuracy: 0.0000e+00 - val_loss: -0.6902 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 185ms/step - loss: -0.7019 - accuracy: 0.0000e+00 - val_loss: -1.1252 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 176ms/step - loss: -1.1382 - accuracy: 0.0000e+00 - val_loss: -1.6942 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 171ms/step - loss: -1.7084 - accuracy: 0.0000e+00 - val_loss: -2.3949 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 158ms/step - loss: -2.4104 - accuracy: 0.0000e+00 - val_loss: -3.1856 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 168ms/step - loss: -3.2025 - accuracy: 0.0000e+00 - val_loss: -4.0024 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 163ms/step - loss: -4.0208 - accuracy: 0

In [128]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    elif (i < 0.5 and i >= -0.5):
       pred_labels.append(0)
    else:
        pred_labels.append(-1)
print(len(test_labels))
print(pred_labels)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

1/1 [==============================] - 2s 2s/step
1341
[0, 0, 0, 0, 0]


ValueError: Found input variables with inconsistent numbers of samples: [1341, 5]